In [58]:
import cv2
import mediapipe as mp
import numpy as np
import os
import sounddevice as sd
from scipy.io.wavfile import write
import threading
import time
import re


In [ ]:
#Do not run
# === CONFIG ===
DATASET_PATH = 'D:\minor_project\Shruti---AVSR-in-Nepali-Language-\MajorProject'
FRAMES_PATH = os.path.join(DATASET_PATH, 'frames')
AUDIO_PATH = os.path.join(DATASET_PATH, 'audio')
SEQUENCE_ID = 0   # Increment manually or via loop for multiple sequences
N_FRAMES = 100
SR = 16000  # Audio sampling rate
CROP_SIZE = 150  # Size of the square crop

LIPS_IDX = sorted(set([0,57, 185, 40, 39, 37, 267, 269, 270, 409, 310]))
CENTER_IDX = LIPS_IDX[0]  # Use first lip landmark to center crop

# Create directories
os.makedirs(FRAMES_PATH, exist_ok=True)
os.makedirs(AUDIO_PATH, exist_ok=True)
for action in actions:
    for sequence in range(no_sequences):
        os.makedirs(os.path.join(DATASET_PATH, action, str(sequence)), exist_ok=True)

In [ ]:
#Run this STEP-01
DATA_PATH = 'D://MajorProject'
AUDIO_PATH = os.path.join(DATA_PATH, 'audio')
FRAME_PATH = os.path.join(DATA_PATH, 'frames')

actions = np.array(['Paschim'])  # Modify as needed
no_sequences = 120
sequence_length = 80
SR = 16000  # Audio sample rate
CROP_SIZE = 120  # Square size for lip crop

LIPS_IDX =  sorted(set([0,57, 185, 40, 39, 37, 267, 269, 270, 409, 310]))
CENTER_IDX = LIPS_IDX[0]  # Use first lip landmark to center crop

# Make sure base directories exist
os.makedirs(FRAME_PATH, exist_ok=True)
os.makedirs(AUDIO_PATH, exist_ok=True)

mp_face_mesh = mp.solutions.face_mesh

def record_audio(duration, audio_buffer):
    print("🎙 Audio recording started...")
    audio = sd.rec(int(duration * SR), samplerate=SR, channels=1, dtype='int16')
    sd.wait()
    audio_buffer.append(audio)
    print("🎙 Audio recording completed.")



In [ ]:
# === AUDIO RECORDING ===
# ==== Do not Run this === 
audio_recording = []

def record_audio(duration):
    global audio_recording
    print("🎙 Audio recording started...")
    audio_recording = sd.rec(int(duration * SR), samplerate=SR, channels=1, dtype='int16')
    sd.wait()
    print("🎙 Audio recording completed.")

# === VIDEO CAPTURE ===
cap = cv2.VideoCapture(0)
mp_face_mesh = mp.solutions.face_mesh

with mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1,
                           min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    capturing = False
    saved_frames = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(image_rgb)

        display_frame = frame.copy()

        if results.multi_face_landmarks:
            landmarks = results.multi_face_landmarks[0]
            h, w, _ = frame.shape
            center_lm = landmarks.landmark[CENTER_IDX]
            cx, cy = int(center_lm.x * w), int(center_lm.y * h)

            # Draw center point
            cv2.circle(display_frame, (cx, cy), 3, (0, 255, 0), -1)

            # Compute crop
            x1 = max(cx - CROP_SIZE // 2, 0)
            y1 = max(cy - CROP_SIZE // 2, 0)
            x2 = min(cx + CROP_SIZE // 2, w)
            y2 = min(cy + CROP_SIZE // 2, h)

            cropped = frame[y1:y2, x1:x2]
            gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)

            # If capturing, save frames
            if capturing and saved_frames < N_FRAMES:
                seq_path = os.path.join(FRAMES_PATH, f'seq_{SEQUENCE_ID}')
                os.makedirs(seq_path, exist_ok=True)
                cv2.imwrite(os.path.join(seq_path, f'{saved_frames}.jpg'), gray)

                # Show cropped frame in window
                cv2.putText(display_frame, f'Capturing Frame {saved_frames+1}/{N_FRAMES}', 
                            (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                cv2.imshow('Captured Crop', gray)

                saved_frames += 1

                # Stop after N_FRAMES
                if saved_frames == N_FRAMES:
                    capturing = False
                    print("✅ Frame capture completed.")
                    audio_thread.join()
                    write(os.path.join(AUDIO_PATH, f'seq_{SEQUENCE_ID}.wav'), SR, audio_recording)
                    print(f"✅ Audio saved to seq_{SEQUENCE_ID}.wav")

        cv2.imshow('Feedback', display_frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('s') and not capturing:
            # Start capture
            print("🚀 Starting capture...")
            capturing = True
            saved_frames = 0
            audio_thread = threading.Thread(target=record_audio, args=(N_FRAMES / 30,))
            audio_thread.start()
        elif key == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [54]:
#Run this Step -02
def get_next_sequence_number(base_path, action):
    action_path = os.path.join(base_path, action)
    if not os.path.exists(action_path):
        return 0
    entries = os.listdir(action_path)
    numeric_dirs = [int(name) for name in entries if name.isdigit()]
    return max(numeric_dirs) + 1 if numeric_dirs else 0


In [23]:
get_next_sequence_number("D://MajorProject//frames",'Anuman')

30

In [55]:
#Main
# Run this step - 3
audio_recording = []
cap = cv2.VideoCapture(0)


with mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1,
                           min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    for action in actions:
        offset=get_next_sequence_number('D://MajorProject//frames',action)
        for sequence in range(no_sequences):
            print(f" Ready for action: {action}, sequence: {sequence}. Press 's' to start.")

            while True:
                ret, frame = cap.read()
                display_frame = frame.copy()
                ret, frame = cap.read()
                display_frame = frame.copy()
                image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = face_mesh.process(image_rgb)

                cv2.putText(display_frame, f"Press 's' to start {action} {sequence}", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                cv2.imshow('OpenCV Feed', display_frame)

                if results.multi_face_landmarks:
                    landmarks = results.multi_face_landmarks[0]
                    h, w, _ = frame.shape
                    center_lm = landmarks.landmark[CENTER_IDX]
                    cx, cy = int(center_lm.x * w), int(center_lm.y * h)

                    x1 = max(cx - CROP_SIZE // 2, 0)
                    y1 = max(cy - CROP_SIZE // 2, 0)
                    x2 = min(cx + CROP_SIZE // 2, w)
                    y2 = min(cy + CROP_SIZE // 2, h)

                    # Draw crop rectangle
                    cv2.rectangle(display_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

                key = cv2.waitKey(10) & 0xFF
                if key == ord('s'):
                   
                    #time.sleep(3)
                    break
                elif key == ord('q'):
                    cap.release()
                    cv2.destroyAllWindows()
                    exit()

           
            seq_frame_path = os.path.join(FRAME_PATH, action, str(offset+sequence))
            os.makedirs(seq_frame_path, exist_ok=True)
            action_audio_path = os.path.join(AUDIO_PATH, action)
            os.makedirs(action_audio_path, exist_ok=True)

            audio_buffer = []
            audio_thread = threading.Thread(target=record_audio, args=(sequence_length / 30, audio_buffer))
            audio_thread.start()

            saved_frames = 0
            prev_time = time.time()

            while saved_frames < sequence_length:
                ret, frame = cap.read()
                display_frame = frame.copy()
                image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = face_mesh.process(image_rgb)

                # FPS calculation
                curr_time = time.time()
                fps = 1 / (curr_time - prev_time)
                prev_time = curr_time

                if results.multi_face_landmarks:
                    landmarks = results.multi_face_landmarks[0]
                    h, w, _ = frame.shape
                    center_lm = landmarks.landmark[CENTER_IDX]
                    cx, cy = int(center_lm.x * w), int(center_lm.y * h)

                    x1 = max(cx - CROP_SIZE // 2, 0)
                    y1 = max(cy - CROP_SIZE // 2, 0)
                    x2 = min(cx + CROP_SIZE // 2, w)
                    y2 = min(cy + CROP_SIZE // 2, h)

                    # Draw crop rectangle
                    cv2.rectangle(display_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

                    crop = frame[y1:y2, x1:x2]
                    gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)

                    # Save cropped grayscale frame
                    frame_path = os.path.join(seq_frame_path, f'{saved_frames}.jpg')
                    cv2.imwrite(frame_path, gray)

                    saved_frames += 1

                # Draw FPS
                cv2.putText(display_frame, f'FPS: {fps:.2f}', (10, 70),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)

                # Draw progress bar
                bar_length = int((saved_frames / sequence_length) * (w - 20))
                cv2.rectangle(display_frame, (10, h - 20), (10 + bar_length, h - 10), (0, 255, 0), -1)
                cv2.rectangle(display_frame, (10, h - (2*10)), (w - 10, h - 10), (255, 255, 255), 2)

                # Show windows
                cv2.imshow('OpenCV Feed', display_frame)
                cv2.imshow('Cropped Lips', gray if results.multi_face_landmarks else np.zeros((CROP_SIZE, CROP_SIZE), dtype=np.uint8))

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    cap.release()
                    cv2.destroyAllWindows()
                    exit()

            audio_thread.join()
            write(os.path.join(action_audio_path, f'{offset+sequence}.wav'), SR, audio_buffer[0])
            print(f"✅ Saved audio for {action} sequence {sequence+offset}")

cap.release()
cv2.destroyAllWindows()

 Ready for action: Namaste, sequence: 0. Press 's' to start.
🎙 Audio recording started...
🎙 Audio recording completed.
✅ Saved audio for Namaste sequence 125
 Ready for action: Namaste, sequence: 1. Press 's' to start.
🎙 Audio recording started...
🎙 Audio recording completed.
✅ Saved audio for Namaste sequence 126
 Ready for action: Namaste, sequence: 2. Press 's' to start.
🎙 Audio recording started...
🎙 Audio recording completed.
✅ Saved audio for Namaste sequence 127
 Ready for action: Namaste, sequence: 3. Press 's' to start.
🎙 Audio recording started...
🎙 Audio recording completed.
✅ Saved audio for Namaste sequence 128
 Ready for action: Namaste, sequence: 4. Press 's' to start.
🎙 Audio recording started...
🎙 Audio recording completed.
✅ Saved audio for Namaste sequence 129
 Ready for action: Namaste, sequence: 5. Press 's' to start.
🎙 Audio recording started...
🎙 Audio recording completed.
✅ Saved audio for Namaste sequence 130
 Ready for action: Namaste, sequence: 6. Press 's' t

In [17]:
cap.release()
cv2.destroyAllWindows()